<a href="https://colab.research.google.com/github/rhysdavies21/library/blob/master/omega_draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Comparing the Omega Measure across asset class

In [0]:
# Input list of underlying assets, dates and compare historical Omega measure

In [148]:
pip install yfinance --upgrade --no-cache-dir

Requirement already up-to-date: yfinance in /usr/local/lib/python3.6/dist-packages (0.1.49)


In [0]:
# Import libraries
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [0]:
# Choose inputs
assets = '^FTSE ^GSPC HISFX AAPL'
start_date = '2019-01-01'
end_date = '2019-10-24'

# Generate inputs
#assets = str(input("Choose tickers (string format with space between tickers) "))
#start_date = str(input("Choose start date (yyyy-mm-dd) "))
#end_date = str(input("Choose end date (yyyy-mm-dd) "))

In [152]:
# Display inputs
print('assets: ', type(assets), assets)
print('start_date: ', type(start_date), start_date)
print('end_date: ', type(end_date), end_date)

assets:  <class 'str'> ^FTSE ^GSPC HISFX AAPL
start_date:  <class 'str'> 2019-01-01
end_date:  <class 'str'> 2019-10-24


In [153]:
# Import data
data = yf.download(assets, start_date, end_date)

# Review data
print(color.BOLD+'Indexing information:'+color.END)
print(data.index.dtype)
print(type(data.index))
print(data.index)

print(color.BOLD+'Dataframe information:'+color.END)
print('Data Shape: ', data.shape)
print('Data Head', data.head(1))
print('Data Tail', data.tail(1))

[*********************100%***********************]  4 of 4 completed
Indexing information:
datetime64[ns]
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
DatetimeIndex(['2019-01-02', '2019-01-03', '2019-01-04', '2019-01-07',
               '2019-01-08', '2019-01-09', '2019-01-10', '2019-01-11',
               '2019-01-14', '2019-01-15',
               ...
               '2019-10-11', '2019-10-14', '2019-10-15', '2019-10-16',
               '2019-10-17', '2019-10-18', '2019-10-21', '2019-10-22',
               '2019-10-23', '2019-10-24'],
              dtype='datetime64[ns]', name='Date', length=210, freq=None)
Dataframe information:
Data Shape:  (210, 24)
Data Head              Adj Close                      ... Volume                           
                  AAPL  HISFX        ^FTSE  ...  HISFX        ^FTSE         ^GSPC
Date                                        ...                                  
2019-01-02  156.049484  11.46  6734.200195  ...    0.0  614252000.0  3.733

In [154]:
# Change into performance metric
new_df = data['Adj Close'].pct_change(periods=1)   # Example: (data_adjclose['^FTSE'][1] / data_adjclose['^FTSE'][0] ) -1

print('Type', type(new_df))
print('Shape', new_df.shape)
new_df.head(3)

Type <class 'pandas.core.frame.DataFrame'>
Shape (210, 4)


,AAPL,HISFX,^FTSE,^GSPC
Date,,,,
2019-01-02,NaN,NaN,NaN,NaN
2019-01-03,-0.099607,-0.003490,-0.006163,-0.024757
2019-01-04,0.042689,0.024518,0.021621,0.034336


In [155]:
asset_list = list(new_df.columns)[0:]
print('Assets: ', asset_list)

Assets:  ['AAPL', 'HISFX', '^FTSE', '^GSPC']


In [320]:
# calc_omega is function t create omega by asset for a given input of Threshold

def calc_omega(thresh):
  threshold = thresh
  update_new_df = new_df
  omega_assets = []
  omega_losses = []
  omega_gains = []
  omega_values = []

  for i in asset_list:
    update_new_df['loss_col_'+i]= update_new_df[i].apply(lambda x: float(1) if x < threshold else float(0)) * (threshold - update_new_df[i])
    update_new_df['gain_col_'+i]= update_new_df[i].apply(lambda x: float(1) if x > threshold else float(0)) * (update_new_df[i] - threshold)

    omega_assets.append(i)
    omega_losses.append(sum(update_new_df['loss_col_'+i][1:]))
    omega_gains.append(sum(update_new_df['gain_col_'+i][1:]))
    omega_values.append(omega_gains[asset_list.index(i)] / omega_losses[asset_list.index(i)])

  data_summary = {'assets' : omega_assets, 'losses' : omega_losses, 'gains' : omega_gains, 'omega' : omega_values }
  omega_summary = pd.DataFrame(data_summary)
  
  rd = dict()
  rd['Assets'] = omega_assets
  rd['Omega Values'] = omega_values

 # return print(omega_summary, type(omega_summary)) 
  # return list(omega_values)
  return rd

#calc_omega(0.03)['Omega Values']
calc_omega(0.03)


{'Assets': ['AAPL', 'HISFX', '^FTSE', '^GSPC'],
 'Omega Values': [0.01787319271661638, 0.0, 0.0, 0.000712283716337993]}

In [312]:
display(type(calc_omega(0.01)))
calc_omega(0.01)[2]

# build a dictionary
# https://www.geeksforgeeks.org/g-fact-41-multiple-return-values-in-python/ 

dict

KeyError: ignored

In [159]:
omega_range = [0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]
print('omega_range: ', omega_range)

omega_range:  [0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]


In [0]:
def fun(): 
    d = dict();  
    d['str'] = "GeeksforGeeks"
    d['x']   = 20
    return d 
 
# Driver code to test above method 
d = fun()  
print(d) 

In [307]:
# Driver code to test above method 
d = fun()  
print(d) 


{'str': 'GeeksforGeeks', 'x': 20}
